# PoseEstimation_Matching

Con il modulo "sys" posso importare la directory di "models" allo stesso livello del PoseEstimation_Utils.py, così da richiamare la funzione di utils.py e matching.py

In [1]:
import sys
sys.path.append('./SuperGluePretrainedNetwork/models')
import utils
import matching

from PIL import Image
import torchvision.transforms as transforms
import torch

Richiamo l'oggetto dalla classe necessaria per performare il matching, cambiando l'attributo dei weights da indoor ad outdoor

In [2]:
match = matching.Matching()
match.weights = "outdoor"
#print(match.weights)

Loaded SuperPoint model
Loaded SuperGlue model ("indoor" weights)


C:\Users\antoa\ImageClassification\SuperGluePretrainedNetwork/models\superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str(pat

Definisco la funzione di matching che runna Superpoint e SuperGlue sulle due immagini scelte, a valle della conversione delle stesse nel formato accettato

In [11]:
ZeroImg_path = "./SuperGluePretrainedNetwork/MoonLu/MoonImg/Nadir.png"
FirstImg_path = "./SuperGluePretrainedNetwork/MoonLu/MoonImg/pitch_15_deg.png"

ZeroImg = Image.open(ZeroImg_path).convert("L") #Il comando .convert("L") serviva per convertire l'immagine in scala di grigi, non accettando RGB
FirstImg = Image.open(FirstImg_path).convert("L")

transform = transforms.ToTensor() #Funzione utile a convertire l'immagine in Tensore, unico tipo di dato accettato dalla funzine di matching

ZeroTensor = transform(ZeroImg)
FirstTensor = transform(FirstImg)

#.unsqueeze(1) permette di aggiungere un valore al vettore che la funzione va a "unpackettare", infatti
# si aspetta (batch_size, channels, height, width), ma dalla foto in scala di grigi ottiene solo (batch_size, height, width).
# Ovvero [1, 1024, 1024] invece che [1, 1, 1024, 1024]
ZeroTensor = ZeroTensor.unsqueeze(1) 
FirstTensor = FirstTensor.unsqueeze(1)

Image_to_Compare = {"image0": ZeroTensor , #la funzione accetta un dictionary in input con le due immagini
                    "image1": FirstTensor}
#print(ZeroTensor.shape)
match.forward(Image_to_Compare) #Funzione di matching

{'keypoints0': [tensor([[ 179.,    8.],
          [ 315.,    8.],
          [ 505.,    8.],
          ...,
          [ 266., 1015.],
          [ 522., 1015.],
          [ 763., 1015.]])],
 'scores0': (tensor([0.0060, 0.0386, 0.0348,  ..., 0.0203, 0.0238, 0.0221],
         grad_fn=<IndexBackward0>),),
 'descriptors0': [tensor([[ 3.2384e-02,  2.9950e-02,  4.8208e-02,  ...,  2.0744e-02,
           -2.1598e-02, -4.6335e-05],
          [-9.0647e-02,  2.9038e-02, -4.5789e-02,  ...,  1.5710e-02,
           -5.7257e-02, -1.1181e-01],
          [-4.5147e-02,  4.0723e-02,  1.9220e-02,  ...,  2.2097e-01,
            6.1778e-02,  9.2101e-02],
          ...,
          [ 4.3642e-02, -8.7636e-03, -3.0826e-02,  ..., -3.9260e-02,
            1.9090e-02, -4.9287e-02],
          [ 9.7608e-03, -4.5511e-02,  2.1109e-03,  ..., -4.8191e-02,
           -4.8286e-02, -2.3080e-02],
          [-6.3288e-02, -8.0547e-02, -2.8777e-03,  ...,  4.4758e-02,
            1.4759e-02, -1.4621e-03]], grad_fn=<SelectBackward0